In [1]:
!pip install pandas openpyxl requests


In [5]:
import requests
import json
from IPython.display import Image, display
import base64
import os
import random

# 🟡 SETUP
RUNPOD_API_URL = "https://api.runpod.ai/v2/yy1rao0wwxtzxx/runsync"
API_KEY = ""  # 🔐 Replace with your actual RunPod API key



OUTPUT_FOLDER = "output"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# 🔤 ONE INPUT PROMPT
prompt_text = "A futuristic city at sunset with flying cars"

# ------------------- WORKFLOW -------------------
def build_workflow(prompt_text, seed):
    return {
        "3": {
            "inputs": {
                "seed": seed,
                "steps": 20,
                "cfg": 8.1,
                "sampler_name": "euler",
                "scheduler": "normal",
                "denoise": 1,
                "model": ["4", 0],
                "positive": ["6", 0],
                "negative": ["7", 0],
                "latent_image": ["5", 0]
            },
            "class_type": "KSampler"
        },
        "4": {
            "inputs": {
                "ckpt_name": "dynavisionXLAllInOneStylized_releaseV0610Bakedvae.safetensors"
            },
            "class_type": "CheckpointLoaderSimple"
        },
        "5": {
            "inputs": {
                "width": 512,
                "height": 512,
                "batch_size": 1
            },
            "class_type": "EmptyLatentImage"
        },
        "6": {
            "inputs": {
                "text": prompt_text,
                "clip": ["4", 1]
            },
            "class_type": "CLIPTextEncode"
        },
        "7": {
            "inputs": {
                "text": "text, watermark",
                "clip": ["4", 1]
            },
            "class_type": "CLIPTextEncode"
        },
        "8": {
            "inputs": {
                "samples": ["3", 0],
                "vae": ["4", 2]
            },
            "class_type": "VAEDecode"
        },
        "9": {
            "inputs": {
                "filename_prefix": "ComfyUI",
                "images": ["8", 0]
            },
            "class_type": "SaveImage"
        }
    }

# ------------------- GENERATE AND SAVE -------------------
def generate_single_image(prompt_text):
    seed = random.randint(1, 999999999)
    workflow = build_workflow(prompt_text, seed)

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "input": {
            "prompt": workflow
        }
    }

    print(f"🚀 Generating image with prompt:\n{prompt_text}")
    try:
        response = requests.post(RUNPOD_API_URL, json=payload, headers=headers)
        response.raise_for_status()
        data = response.json()
        output = data.get("output", {})
        image_url = output.get("image_url") or output.get("images", [None])[0]

        if not image_url:
            print("⚠️ No image returned.")
            return

        if image_url.startswith("http"):
            image_data = requests.get(image_url).content
        else:
            image_data = base64.b64decode(image_url)

        save_path = os.path.join(OUTPUT_FOLDER, "single_output.png")
        with open(save_path, "wb") as f:
            f.write(image_data)

        print(f"✅ Image saved to {save_path}")

    except Exception as e:
        print(f"❌ Error generating image: {e}")

# ------------------- RUN -------------------
generate_single_image(prompt_text)


🚀 Generating image with prompt:
A futuristic city at sunset with flying cars
✅ Image saved to output/single_output.png


In [ ]:
import pandas as pd
import requests
import base64
import os
import random
from tqdm import tqdm

# ------------------- CONFIG -------------------
API_KEY = "..."  # 🔐 Replace this!
RUNPOD_API_URL = "https://api.runpod.ai/v2/yy1rao0wwxtzxx/runsync"
CSV_FILE = "/content/Teacher,Student.csv"  # 📄 Your CSV file path
OUTPUT_FOLDER = "output"                   # 📁 Where to save images
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ------------------- WORKFLOW BUILDER -------------------
def build_workflow(prompt_text, seed):
    return {
        "3": {
            "inputs": {
                "seed": seed,
                "steps": 20,
                "cfg": 8.1,
                "sampler_name": "euler",
                "scheduler": "normal",
                "denoise": 1,
                "model": ["4", 0],
                "positive": ["6", 0],
                "negative": ["7", 0],
                "latent_image": ["5", 0]
            },
            "class_type": "KSampler"
        },
        "4": {
            "inputs": {
                "ckpt_name": "dynavisionXLAllInOneStylized_releaseV0610Bakedvae.safetensors"
            },
            "class_type": "CheckpointLoaderSimple"
        },
        "5": {
            "inputs": {
                "width": 512,
                "height": 512,
                "batch_size": 1
            },
            "class_type": "EmptyLatentImage"
        },
        "6": {
            "inputs": {
                "text": prompt_text,
                "clip": ["4", 1]
            },
            "class_type": "CLIPTextEncode"
        },
        "7": {
            "inputs": {
                "text": "text, watermark",
                "clip": ["4", 1]
            },
            "class_type": "CLIPTextEncode"
        },
        "8": {
            "inputs": {
                "samples": ["3", 0],
                "vae": ["4", 2]
            },
            "class_type": "VAEDecode"
        },
        "9": {
            "inputs": {
                "filename_prefix": "ComfyUI",
                "images": ["8", 0]
            },
            "class_type": "SaveImage"
        }
    }

# ------------------- GENERATE + SAVE IMAGE -------------------
def generate_and_save_image(prompt, row_idx, role):
    seed = random.randint(1, 999999999)
    workflow = build_workflow(prompt, seed)

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "input": {
            "prompt": workflow
        }
    }

    print(f"🚀 Generating image (row {row_idx+1}, {role})...")
    try:
        response = requests.post(RUNPOD_API_URL, json=payload, headers=headers)
        response.raise_for_status()
    except Exception as e:
        print(f"❌ Request failed: {e}")
        return

    data = response.json()
    output = data.get("output", {})
    image_url = output.get("image_url") or output.get("images", [None])[0]

    if not image_url:
        print(f"⚠️ No image returned for row {row_idx+1} - {role}")
        return

    try:
        if image_url.startswith("http"):
            image_data = requests.get(image_url).content
        else:
            image_data = base64.b64decode(image_url)

        filename = os.path.join(OUTPUT_FOLDER, f"row_{row_idx+1}_{role}.png")
        with open(filename, "wb") as f:
            f.write(image_data)

        print(f"✅ Saved: {filename}")
    except Exception as e:
        print(f"❌ Failed to save image for row {row_idx+1} - {role}: {e}")

# ------------------- PROCESS CSV -------------------
def process_csv():
    try:
        df = pd.read_csv(CSV_FILE)
    except Exception as e:
        print(f"❌ Failed to load CSV file: {e}")
        return

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating images"):
        for role in ["Teacher", "Student"]:
            prompt = row.get(role)
            if isinstance(prompt, str) and prompt.strip():
                generate_and_save_image(prompt, idx, role)
            else:
                print(f"⚠️ Empty or invalid prompt at row {idx+1} - {role}")

# ------------------- RUN -------------------
if __name__ == "__main__":
    process_csv()


❌ Failed to load Excel file: Excel file format cannot be determined, you must specify an engine manually.
